In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json


# Point this path to where you unzipped the clinical_trial_risk repository
REPO_DIR = "/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/projects_client/clinical_trial_risk"
sys.path.append(REPO_DIR + "/front_end")

df_annotations = pd.read_csv("../../data/ctgov/annotations/all_annotations.csv")


In [2]:
MODELS_DIR = REPO_DIR + '/front_end/models/'

In [3]:
from util.page_tokeniser import tokenise_pages

In [4]:
from processors.num_subjects_extractor import NumSubjectsExtractor

/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2022-11-21 23:52:35.549549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 23:52:35.702180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file:

In [5]:
from processors.num_subjects_extractor_naive_bayes import NumSubjectsExtractorNaiveBayes

In [6]:
num_subjects_extractor = NumSubjectsExtractor(MODELS_DIR + "num_subjects_classifier.pkl.bz2")

/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
num_subjects_extractor_nb = NumSubjectsExtractorNaiveBayes(MODELS_DIR + "subjects_classifier_document_level.pkl.bz2")

Initialising Num Subjects classifier /media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/projects_client/clinical_trial_risk/front_end/models/subjects_classifier_document_level.pkl.bz2


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/base.py:32

In [8]:
df_train = df_annotations[df_annotations.train_val == "val"]

In [9]:
df_train_small = df_train[~df_train.num_subjects.isna()].sample(1000)

In [10]:
texts =[]
for i in range(len(df_train_small)):
    file_name = df_train_small.file.iloc[i]

    with open("/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/data_open/read_ctgov_data/json/" + file_name + ".json", "r", encoding="utf-8") as f:
        pages = json.load(f)
    texts.append(pages)
df_train_small["pages"] = texts

In [11]:
pred1 = []
pred2 = []


y1 = []
y2 = []
for idx in range(len(df_train_small)):
    tokenised_pages = tokenise_pages(df_train_small.pages.iloc[idx])
    
    subj_to_pages = num_subjects_extractor.process(tokenised_pages)
    pred1.append(subj_to_pages["prediction"])
    y1.append(subj_to_pages["proba"])
    
    subj_to_pages_nb = num_subjects_extractor_nb.process(tokenised_pages)
    pred2.append(subj_to_pages_nb["prediction"])
    y2.append(subj_to_pages_nb["proba"])


In [12]:
df_train_small["y1"] = y1
df_train_small["y2"] = y2


In [13]:
df_train_small["y1_pred"] = pred1
df_train_small["y2_pred"] = pred2


In [14]:
def get_num_subjects_clean(num):
    if num >= 134:
        return "134+"
    if num >= 34:
        return "34-133"
    return "1-33"

In [15]:
df_train_small

,file,nct_id,train_val,num_arms,num_subjects,phase,intervention_type,country,has_sap,pages,y1,y2,y1_pred,y2_pred
8489,71_NCT02862171_Prot_002.pdf,NCT02862171,val,1.0,941.0,Phase 3,Drug,"ZA,UG",0,[\n \n\nIPM 032 \n\nProtocol \n \n\nCONFIDENTI...,{'1400': 0.6},"{'1-33': 0.22776806102175343, '134+': 0.448837...",1400,134+
1128,09_NCT02892409_Prot_001.pdf,NCT02892409,val,2.0,30.0,Phase 1,Drug,KR,0,"[\nThis may include, but is not limited to, re...","{'30': 0.51, '15': 0.17, '14': 0.11}","{'1-33': 0.4602939399120555, '134+': 0.2011895...",30,1-33
3082,25_NCT04805125_Prot_SAP_000.pdf,NCT04805125,val,2.0,610.0,Phase 3,Biological,CH,1,[\nClinical Study first sub-protocol \n \n\nRa...,{'380': 0.3},"{'1-33': 0.09782174395854197, '134+': 0.687297...",380,134+
3194,26_NCT03877926_Prot_000.pdf,NCT03877926,val,4.0,3862.0,Phase 3,Biological,US,0,[\n \n \n\n \n\n \n\n \n \n\n ...,{},"{'1-33': 0.32438384897745165, '134+': 0.331620...",0,34-133
11575,97_NCT01885897_Prot_SAP_000.pdf,NCT01885897,val,1.0,33.0,Phase 1/Phase 2,Biological,US,1,"[\n \n \n\nMasonic Cancer Center, University o...","{'31': 0.22, '14': 0.2, '16': 0.16}","{'1-33': 0.7763629032056141, '134+': 0.0100682...",31,1-33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11725,98_NCT02818998_Prot_000.pdf,NCT02818998,val,3.0,463.0,Phase 3,Drug,"AT,CA,CZ,FR,DE,HU,IT,LT,PL,PT,SK,ES,CH,GB,CZ",0,[\n \n\nDocument Type: Study Protocol \n\nOffi...,{'490': 0.39},"{'1-33': 0.09625335414258913, '134+': 0.686142...",490,134+
1616,13_NCT02856113_Prot_000.pdf,NCT02856113,val,2.0,152.0,Phase 3,Drug,"US,BR,IL,IT,MX,PL,RU,AR,DE,HU",0,"[\nC\nC\nI\n\n, \n1.0 TITLE PAGE\n\nPROTOCOL\n...","{'75': 0.42, '100': 0.14, '150': 0.14}","{'1-33': 0.16220942469473512, '134+': 0.636224...",75,134+
637,05_NCT02853305_Prot_SAP_000.pdf,NCT02853305,val,3.0,1010.0,Phase 3,BiologicalDrug,"AR,BE,BR,CA,CL,FR,DE,HU,IE,IL,JP,KR,NL,RU,ZA,E...",1,[\nOfficial Protocol Title: \n\nNCT number: \n...,"{'990': 0.64, '1300': 0.08}","{'1-33': 0.36412900321960673, '134+': 0.298009...",990,1-33
1996,16_NCT03802916_Prot_SAP_002.pdf,NCT03802916,val,2.0,30.0,Phase 2,Drug,"US,CA,GR,IT",1,[\n \n\n \n CONFIDENTIAL Page 1 of 58 \n \n \n...,"{'30': 0.29, '28': 0.15, '15': 0.12}","{'1-33': 0.33955007029632955, '134+': 0.246443...",30,34-133


In [17]:
probas3 = []
for idx in range(len(df_train_small)):
    combined_probas = {}
    rule_based = df_train_small["y1"].iloc[idx]
    
    for i, p in rule_based.items():
        # Multiply by number of arms if applicable
#         if i in num_subjects_to_pages["is_per_arm"]:
#             i *= num_arms_to_pages["prediction"]
        cat = get_num_subjects_clean(int(i))
        combined_probas[i] = (p + df_train_small["y2"].iloc[idx][cat]) / 2
    
    
    probas3.append(combined_probas)
df_train_small["y3"] = probas3

In [18]:
df_train_small["y3_pred"] = df_train_small["y3"].apply(lambda x : max(x, key=x.get) if len(x) > 0 else 0)

In [22]:
df_train_small["y3_pred"].iloc[0]

'1400'

In [25]:
df_train_small["num_subjects"].iloc[0]

941.0

In [24]:
df_train_small["y1"].iloc[0]

{'1400': 0.6}

In [23]:
df_train_small["y3"].iloc[0]

{'1400': 0.5244186935010103}

In [19]:
accuracy_score( df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)), df_train_small.y1_pred.apply(str))

0.123

In [20]:
accuracy_score( df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)), df_train_small.y2_pred.apply(str))

0.0

In [21]:
accuracy_score( df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)), df_train_small.y3_pred.apply(str))

0.122

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay.from_predictions(df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)), 
                                        df_train_small.y1_pred.apply(str))
plt.xticks(rotation=90)
;

In [ ]:
pd.DataFrame({"gt":df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)),
             "pred":df_train_small.y1_pred.apply(str)
             })

In [ ]:
df_train_small.file

In [ ]:
pd.DataFrame({"gt":df_train_small.num_subjects.apply(str).apply(lambda x : re.sub(r'\.0', '', x)),
             "pred":df_train_small.y3_pred.apply(str)
             })

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_train_small.num_arms_clean.value_counts()

In [ ]:
plt.scatter(df_train_small.y1, df_train_small.num_arms_clean)

In [ ]:
plt.scatter(df_train_small.y2, df_train_small.num_arms_clean)

In [ ]:
from sklearn.metrics import roc_auc_score
for model in 2, 3, 4:
    all_aucs = []
    for num_arms in ["1", "2", "3+"]:
        roc_auc = roc_auc_score(df_train_small.num_arms_clean == num_arms, df_train_small[f"y{model}"].apply(lambda m : m[num_arms]))
        print (f"SAP ROC AUC {roc_auc} for model {model} arms {num_arms}")
        all_aucs.append(roc_auc)
    print (f"Average model {model}", np.mean(all_aucs))

In [ ]:
plt.figure()
lw = 2
    
for model in 2, 3, 4:
    for num_arms in ["1", "2", "3+"]:
        fpr, tpr, _ = roc_curve(df_train_small.num_arms_clean==num_arms,df_train_small[f"y{model}"].apply(lambda m : m[num_arms]))
        roc_auc = auc(fpr, tpr)
        plt.plot(
            fpr,
            tpr,
            # color="darkorange",
            lw=lw,
            label=f"ROC curve model {model} {num_arms} arms (area = {roc_auc:0.2f})"
        )
    plt.plot([0, 1], [0, 1], color="gray", alpha=0.5, lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()